In [ ]:
from keras.layers import Flatten,Dense,Input,Lambda
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator 
import numpy as np 
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input 
import glob
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [ ]:
IMAGE_SIZE = (224,224,3)

In [1]:
from google.colab import drive 
drive.mount('gdrive')

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


In [ ]:
train_path = '/content/gdrive/MyDrive/chest_xray/train/'
test_path = '/content/gdrive/MyDrive/chest_xray/test/'

In [ ]:
# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False
  



In [ ]:
  
  # useful for getting number of classes
#folders = glob('/content/gdrive/MyDrive/chest_xray/train/*')
  

# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(2, activation='softmax')(x)



In [ ]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()



In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:



from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)




In [ ]:
training_set = train_datagen.flow_from_directory('/content/gdrive/MyDrive/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/gdrive/MyDrive/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 2961 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
callbacks = [ 
    EarlyStopping(monitor = 'loss', patience = 7), 
    ReduceLROnPlateau(monitor = 'loss', patience = 4), 
    ModelCheckpoint('/content/gdrive/MyDrive/chest_xray/vgg.hdf5', monitor='loss' , save_best_only=True) # saving the best model
]

In [ ]:
'''r=model.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 5,
                         validation_data = test_set,
                         nb_val_samples = 2000)'''

# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  callbacks=callbacks
)


In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')



In [ ]:
# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')




In [ ]:



model.save('/content/gdrive/MyDrive/chest_xray/vgg.h5')

In [ ]:
from keras.models import load_model
model = load_model('/content/gdrive/MyDrive/chest_xray/vgg.hdf5')

In [ ]:
def display_random_image(class_names, images, labels):
   
    
    index = np.random.randint(images.shape[0])
    plt.figure()
    plt.imshow(images[index])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.title('Image #{} : '.format(index) + class_names[labels[index]])
    plt.show()

In [ ]:
import numpy as np 
import cv2
import os
from keras.preprocessing import image
test = image.load_img('/content/gdrive/MyDrive/chest_xray/train/PNEUMONIA/person1000_bacteria_2931.jpeg')
test = np.asarray(test)
test = cv2.resize(test, (224,224))
test = test.reshape(1,224,224,3)
result = model.predict(test)
pred_labels = np.argmax(result, axis = 1)
class_names = ['Normal' , 'Pneumonia']
display_random_image(class_names, test, pred_labels)

In [ ]:
def read_data(data_paths):
    for data_path in data_paths:
        labels = ['PNEUMONIA', 'NORMAL']
        images = []
        y = []
        for label in labels:
            curr_path = data_path + label
            for img in os.listdir(curr_path):
                if ('DS' not in img):
                    image_path = os.path.join(curr_path, img)
                    image =  cv2.resize(cv2.imread(image_path), (224,224))
                    if image is not None:
                        images.append([image, label])
                
    images = np.asarray(images)
    return images

In [ ]:


train = read_data([train_path])
test = read_data([test_path])



In [ ]:
def lung_condition(label):
    if label == 'NORMAL':
        return 0
    else:
        return 1

In [ ]:
def splitdata(data):
    X = []
    y = []
    for i, (val, label) in enumerate(data):
        X.append(val)
        y.append(lung_condition(label))
    return np.array(X), np.array(y)

In [ ]:
X_train, y_train = splitdata(training_set)
X_test, y_test = splitdata(test_set)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, result)
print('Confusion Matrix\n')
print(confusion)

In [ ]:
pip install streamlit

In [3]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [3]:
import tensorflow as tf 
from keras.models import load_model 
model = load_model('/content/gdrive/MyDrive/chest_xray/vgg.hdf5')

In [2]:
import streamlit as st 
st.write(""" 
               # X-RAY Image Classification 
""")

st.write("This web page is to upload and predict whether the person has pneumonia or not from the patient's chest x-ray image")

file = st.file_uploader("Upload a file", type = ["jpg", "png"])

2021-08-16 05:28:02.375 
  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]


In [4]:
import cv2 
import numpy as np 
from PIL import Image, ImageOps 

def import_predict(image_data, model) :
  size = (224,224) 
  image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
  image = np.asarray(image)
  img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  img_resize = (cv2.resize(img, dsize = (224,224), interpolation = cv2.INTER_CUBIC))/255
  img_reshape = img_resize.reshape(1,224,224,3)
  prediction = model.predict(img_reshape)
  return prediction 






In [5]:
if file is None : 
  st.text("Please upload an image")

else : 
  image = Image.open(file)
  st.image(image, use_column_width = True) 
  prediction = import_predict(image, model)

  if np.argmax(prediction) == 0 : 
    st.write("ITS NORMAL") 
  
  elif np.argmax(prediction) == 1 : 
    st.write("Patient has Pneumonia")

  st.text("Probability (0: Normal, 1:Pneumonia")
  st.write(prediction)
